In [2]:
import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from scipy import spatial

import pickle

from transformers import TFAutoModel, AutoTokenizer

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage, dendrogram

from tqdm import tqdm

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
df = pd.read_pickle('df_km.pickle')

In [11]:
df.sample()

,id,name,embeding,km_30,km_60,km_90,km_120,km_150,km_180,km_210,...,km_720,km_750,km_780,km_810,km_840,km_870,km_900,km_930,km_960,km_990
29629,19242,Минестроне примавера,"[-0.43742597103118896, 0.16899362206459045, -0...",24,0,23,82,45,6,130,...,254,192,149,554,602,799,537,206,137,655


In [4]:
def load_km(number):
    km = pickle.load(open(f"models/kmeans_{number}_cl.pkl", "rb"))
    return km

In [5]:
# обработка текста
def pipline(text, number):
    lem = []
    pattern = r'[^a-zA-Zа-яА-Я0-9 ]'
    text = text.lower()
    text = text.strip()
    text = re.sub(pattern, '', text)
    text = ' '.join(i for i in text.split() if i not in sw)
    for word in text.split():
        p = morph.parse(word)[0]
        lem.append(p.normal_form)
    text = ' '.join(lem)
    tok = tokenizer([text], 
               max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    text = bert(**tok)[1].numpy().tolist()[0]
    km = load_km(number)
    predict = km.predict([text])[0]
    pred = df[df[f'km_{number}'] == predict]
    name = f'km_{number}'
    return pred, name

In [6]:
texts = ['суп', 'шашлык', 'печенье', 'картошка', 'рыба', 'макароны', 'салат']

In [13]:
%%time
# смотрю какое кол-во кластеров больше подходит
res = {}
list_ = []

for number in range(30, 1010, 30):
    subreport = []
    report = []
    true_pred = 0
    for text in texts:
        count = 0
        count_true = 0
        for i in df[['id','name']].iterrows():
            if text in i[1][1].lower():
                count+=1
        res[text] = count
        text1 = f'Всего слово {text} встрречается в df {count} раз'
        print(text1)
        predict, name = pipline(text, number)
        text2 = f'{name} нашел {predict.shape[0]} совпадений'
        print(text2)
        for j in predict[['id','name']].iterrows():
            if text in j[1][1].lower():
                count_true += 1
        text3 = f'из {count} строк в {count_true} действительно есть слово {text}'
        print(text3)
        pred_false = count - count_true
        text4 = f'{pred_false} неправильно предсказаных'
        print(text4)
        print('\n','*' * 50,'\n')
        true_pred += count_true
        
        
        subreport.append(text1)
        subreport.append(text2)
        subreport.append(text3)
        subreport.append(text4)
        subreport.append('\n\n')
        
#         report = '\n'.join(subreport)
        with open("report.txt", "a") as myfile:
            myfile.write('\n'.join(subreport))
    print('\n','*' * 50,'\n')
    print(f'всего правильно предсказано:  {true_pred}')
    print('\n','*' * 50,'\n')
    res[name] = true_pred
sort_dict = sorted(res.items(), key=lambda kv: kv[1], reverse=True)
print(sort_dict)
with open("true_pred.txt", "a") as myfile:
    myfile.write(str(sort_dict))


Всего слово суп встрречается в df 1935 раз
km_30 нашел 1327 совпадений
из 1935 строк в 19 действительно есть слово суп
1916 неправильно предсказаных

 ************************************************** 

Всего слово шашлык встрречается в df 138 раз
km_30 нашел 1327 совпадений
из 138 строк в 5 действительно есть слово шашлык
133 неправильно предсказаных

 ************************************************** 

Всего слово печенье встрречается в df 585 раз
km_30 нашел 510 совпадений
из 585 строк в 177 действительно есть слово печенье
408 неправильно предсказаных

 ************************************************** 

Всего слово картошка встрречается в df 72 раз
km_30 нашел 975 совпадений
из 72 строк в 4 действительно есть слово картошка
68 неправильно предсказаных

 ************************************************** 

Всего слово рыба встрречается в df 250 раз
km_30 нашел 729 совпадений
из 250 строк в 9 действительно есть слово рыба
241 неправильно предсказаных

 ***************************

Всего слово печенье встрречается в df 585 раз
km_180 нашел 121 совпадений
из 585 строк в 59 действительно есть слово печенье
526 неправильно предсказаных

 ************************************************** 

Всего слово картошка встрречается в df 72 раз
km_180 нашел 111 совпадений
из 72 строк в 13 действительно есть слово картошка
59 неправильно предсказаных

 ************************************************** 

Всего слово рыба встрречается в df 250 раз
km_180 нашел 156 совпадений
из 250 строк в 51 действительно есть слово рыба
199 неправильно предсказаных

 ************************************************** 

Всего слово макароны встрречается в df 46 раз
km_180 нашел 196 совпадений
из 46 строк в 0 действительно есть слово макароны
46 неправильно предсказаных

 ************************************************** 

Всего слово салат встрречается в df 2537 раз
km_180 нашел 188 совпадений
из 2537 строк в 178 действительно есть слово салат
2359 неправильно предсказаных

 *****************

Всего слово рыба встрречается в df 250 раз
km_330 нашел 66 совпадений
из 250 строк в 8 действительно есть слово рыба
242 неправильно предсказаных

 ************************************************** 

Всего слово макароны встрречается в df 46 раз
km_330 нашел 132 совпадений
из 46 строк в 0 действительно есть слово макароны
46 неправильно предсказаных

 ************************************************** 

Всего слово салат встрречается в df 2537 раз
km_330 нашел 99 совпадений
из 2537 строк в 96 действительно есть слово салат
2441 неправильно предсказаных

 ************************************************** 


 ************************************************** 

всего правильно предсказано:  165

 ************************************************** 

Всего слово суп встрречается в df 1935 раз
km_360 нашел 194 совпадений
из 1935 строк в 40 действительно есть слово суп
1895 неправильно предсказаных

 ************************************************** 

Всего слово шашлык встрречается в df 

Всего слово салат встрречается в df 2537 раз
km_480 нашел 105 совпадений
из 2537 строк в 101 действительно есть слово салат
2436 неправильно предсказаных

 ************************************************** 


 ************************************************** 

всего правильно предсказано:  181

 ************************************************** 

Всего слово суп встрречается в df 1935 раз
km_510 нашел 44 совпадений
из 1935 строк в 0 действительно есть слово суп
1935 неправильно предсказаных

 ************************************************** 

Всего слово шашлык встрречается в df 138 раз
km_510 нашел 22 совпадений
из 138 строк в 8 действительно есть слово шашлык
130 неправильно предсказаных

 ************************************************** 

Всего слово печенье встрречается в df 585 раз
km_510 нашел 64 совпадений
из 585 строк в 55 действительно есть слово печенье
530 неправильно предсказаных

 ************************************************** 

Всего слово картошка встрречаетс

Всего слово суп встрречается в df 1935 раз
km_660 нашел 70 совпадений
из 1935 строк в 20 действительно есть слово суп
1915 неправильно предсказаных

 ************************************************** 

Всего слово шашлык встрречается в df 138 раз
km_660 нашел 6 совпадений
из 138 строк в 6 действительно есть слово шашлык
132 неправильно предсказаных

 ************************************************** 

Всего слово печенье встрречается в df 585 раз
km_660 нашел 41 совпадений
из 585 строк в 25 действительно есть слово печенье
560 неправильно предсказаных

 ************************************************** 

Всего слово картошка встрречается в df 72 раз
km_660 нашел 19 совпадений
из 72 строк в 13 действительно есть слово картошка
59 неправильно предсказаных

 ************************************************** 

Всего слово рыба встрречается в df 250 раз
km_660 нашел 47 совпадений
из 250 строк в 10 действительно есть слово рыба
240 неправильно предсказаных

 *****************************

Всего слово печенье встрречается в df 585 раз
km_810 нашел 40 совпадений
из 585 строк в 39 действительно есть слово печенье
546 неправильно предсказаных

 ************************************************** 

Всего слово картошка встрречается в df 72 раз
km_810 нашел 35 совпадений
из 72 строк в 3 действительно есть слово картошка
69 неправильно предсказаных

 ************************************************** 

Всего слово рыба встрречается в df 250 раз
km_810 нашел 49 совпадений
из 250 строк в 15 действительно есть слово рыба
235 неправильно предсказаных

 ************************************************** 

Всего слово макароны встрречается в df 46 раз
km_810 нашел 55 совпадений
из 46 строк в 0 действительно есть слово макароны
46 неправильно предсказаных

 ************************************************** 

Всего слово салат встрречается в df 2537 раз
km_810 нашел 44 совпадений
из 2537 строк в 42 действительно есть слово салат
2495 неправильно предсказаных

 ************************

Всего слово рыба встрречается в df 250 раз
km_960 нашел 23 совпадений
из 250 строк в 0 действительно есть слово рыба
250 неправильно предсказаных

 ************************************************** 

Всего слово макароны встрречается в df 46 раз
km_960 нашел 44 совпадений
из 46 строк в 0 действительно есть слово макароны
46 неправильно предсказаных

 ************************************************** 

Всего слово салат встрречается в df 2537 раз
km_960 нашел 34 совпадений
из 2537 строк в 32 действительно есть слово салат
2505 неправильно предсказаных

 ************************************************** 


 ************************************************** 

всего правильно предсказано:  112

 ************************************************** 

Всего слово суп встрречается в df 1935 раз
km_990 нашел 71 совпадений
из 1935 строк в 56 действительно есть слово суп
1879 неправильно предсказаных

 ************************************************** 

Всего слово шашлык встрречается в df 13

## 90 кластеров показали лучший результат

In [17]:
print(sort_dict)

[('салат', 2537), ('суп', 1935), ('печенье', 585), ('km_90', 438), ('km_180', 307), ('km_120', 291), ('km_30', 285), ('km_60', 271), ('рыба', 250), ('km_210', 246), ('km_150', 238), ('km_300', 232), ('km_240', 225), ('km_360', 216), ('km_450', 205), ('km_390', 204), ('km_420', 203), ('km_480', 181), ('km_270', 178), ('km_330', 165), ('km_810', 165), ('km_630', 150), ('km_750', 147), ('km_720', 145), ('km_510', 142), ('km_540', 142), ('km_570', 142), ('шашлык', 138), ('km_780', 138), ('km_990', 137), ('km_660', 136), ('km_870', 136), ('km_840', 114), ('km_960', 112), ('km_690', 110), ('km_600', 98), ('km_930', 88), ('km_900', 80), ('картошка', 72), ('макароны', 46)]
